In [13]:
import pandas as pd
import pandas_datareader as web
import numpy as np
import talib as ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import panel as pn
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# Backtest

In [14]:
#Select stocks
tickers = ['GOOG', 'AAPL', 'BAC']

In [15]:
#Set time
start = '2020, 9, 27'
end = '2021, 9, 27'

In [28]:
df = web.get_data_yahoo('GOOG', start=start, end=end)
(df['Adj Close'].pct_change() + 1).cumprod().hvplot()

:Curve   [Date]   (Adj Close)

In [16]:
#Select technical analysis indicator
def macd(values):
    macd, macdsignal, macdhist = ta.MACD(values, fastperiod=12, slowperiod=26, signalperiod=9)
    return macd, macdsignal

In [17]:
#Create strategy
class MacdCross(Strategy):
    def init(self):
        self.macd = self.I(macd, self.data.Close)[0]
        self.macdsignal = self.I(macd, self.data.Close)[1]
    def next(self):
        if crossover(self.macd, self.macdsignal):
            self.position.close()
            self.buy()
        elif crossover(self.macd, self.macdsignal):
            self.position.close()
            self.sell()    

In [18]:
#Run backtest and create a dataframe to compare stocks
macd_lst1 = []
macd_lst2 = []
for ticker in tickers:
    stock = web.get_data_yahoo(ticker, start=start, end=end)
    stock = stock.drop(columns=['Volume', 'Adj Close'])
    bt = Backtest(stock, MacdCross, cash=10_000)
    macd_lst1.append(bt)
    stats = bt.run()
    macd_lst2.append(stats)
macd_stats = pd.concat(macd_lst2, axis=1, join='outer')

#Change column names to tickers
macd_dct = {}
for i in range(len(tickers)):
    macd_dct[i] = tickers[i]
macd_stats = macd_stats.rename(columns=macd_dct)

macd_stats

,GOOG,AAPL,BAC
Start,2020-09-28 00:00:00,2020-09-28 00:00:00,2020-09-28 00:00:00
End,2021-09-27 00:00:00,2021-09-27 00:00:00,2021-09-27 00:00:00
Duration,364 days 00:00:00,364 days 00:00:00,364 days 00:00:00
Exposure Time [%],71.428571,81.746032,74.603175
Equity Final [$],15228.200073,11899.450371,14140.810076
Equity Peak [$],15653.850708,12808.27047,14336.100126
Return [%],52.282001,18.994504,41.408101
Buy & Hold Return [%],93.238739,26.452676,79.57658
Return (Ann.) [%],52.282001,18.994504,41.408101
Volatility (Ann.) [%],27.595029,27.713007,32.06772


In [19]:
#Visualie backtest result
for i in range(len(macd_lst1)):
    macd_lst1[i].plot()
    plt.clf()

In [20]:
#Compare returns
macd_stats.loc[['Return [%]', 'Buy & Hold Return [%]']].hvplot.bar()

:Bars   [index,Variable]   (value)

In [21]:
#Compare sharpe ratio
macd_stats.loc['Sharpe Ratio'].hvplot.bar()

:Bars   [index]   (Sharpe Ratio)

In [22]:
#Compare maximum drawdown
macd_stats.loc['Max. Drawdown [%]'].hvplot.bar()

:Bars   [index]   (Max. Drawdown [%])

# Dashboard

In [23]:
def macd_df():
    return macd_stats.iloc[:-2]

def macd_rets():
    return macd_stats.loc[['Return [%]', 'Buy & Hold Return [%]']].hvplot.bar()

def macd_plot_1():
    return macd_lst1[0].plot()

def macd_plot_2():
    return macd_lst1[1].plot()

def macd_plot_3():
    return macd_lst1[2].plot()

def macd_sharpe():
    return macd_stats.loc['Sharpe Ratio'].hvplot.bar()

def macd_max_dd():
    return macd_stats.loc['Max. Drawdown [%]'].hvplot.bar()

In [24]:
stats_row = pn.Row(macd_df())
plot_row = pn.Row(macd_plot_1(), macd_plot_2(), macd_plot_3())
rets_col = pn.Column(macd_rets())
sharpe_col = pn.Column(macd_sharpe())
max_dd_col = pn.Column(macd_max_dd())

bt_tabs = pn.Tabs(('Backtst results', stats_row),
                 ('Visualised Backtest results', plot_row),
                 ('Strategy Returns VS Buy & Hold Returns', rets_col),
                 ('Sharpe Ratio', sharpe_col),
                 ('Maximum Drawdown', max_dd_col))
bt_tabs

Tabs
    [0] Row
        [0] DataFrame(DataFrame)
    [1] Row
        [0] Bokeh(Row)
        [1] Bokeh(Row)
        [2] Bokeh(Row)
    [2] Column
        [0] HoloViews(Bars)
    [3] Column
        [0] HoloViews(Bars)
    [4] Column
        [0] HoloViews(Bars)

In [25]:
bt_tabs = pn.Tabs(('Backtst results', stats_row),
                 ('Visualised Backtest results', plot_row),
                 ('Strategy Returns VS Buy & Hold Returns', rets_col),
                 ('Sharpe Ratio', sharpe_col),
                 ('Maximum Drawdown', max_dd_col))
bt_tabs

Tabs
    [0] Row
        [0] DataFrame(DataFrame)
    [1] Row
        [0] Bokeh(Row)
        [1] Bokeh(Row)
        [2] Bokeh(Row)
    [2] Column
        [0] HoloViews(Bars)
    [3] Column
        [0] HoloViews(Bars)
    [4] Column
        [0] HoloViews(Bars)